<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_5_tabular_synthetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
**Module 7: Generative Adversarial Networks**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 7 Material

* Part 7.1: Introduction to GANS for Image and Data Generation [[Video]](https://www.youtube.com/watch?v=0QnCH6tlZgc&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_1_gan_intro.ipynb)
* Part 7.2: Train StyleGAN3 with your Own Images [[Video]](https://www.youtube.com/watch?v=T-MCludVNn4&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_2_Keras_gan.ipynb)
* Part 7.3: Exploring the StyleGAN Latent Vector [[Video]](https://www.youtube.com/watch?v=s1UQPK2KoBY&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_3_style_gan.ipynb)
* Part 7.4: GANS to Enhance Old Photographs Deoldify [[Video]](https://www.youtube.com/watch?v=ZPewmEu7644&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_4_gan_semi_supervised.ipynb)
* **Part 7.5: GANs for Tabular Synthetic Data Generation** [[Video]](https://www.youtube.com/watch?v=cvCvZKvlvq4&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_5_gan_research.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.
  Running the following code will map your GDrive to ```/content/drive```.

In [1]:
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab


# Part 7.5: GANs for Tabular Synthetic Data Generation

Typically GANs are used to generate images. However, we can also generate tabular data from a GAN. In this part, we will use the Python tabgan utility to create fake data from tabular data. Specifically, we will use the Auto MPG dataset to train a GAN to generate fake cars.  [Cite:ashrapov2020tabular](https://arxiv.org/pdf/2010.00638.pdf)

### Installing Tabgan

Pytorch is the foundation of the tabgan neural network utility. The following code installs the needed software to run tabgan in Google Colab. 

In [2]:
!wget https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
!pip install -r requirements.txt
!pip install tabgan

--2021-12-29 13:26:26--  https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 197 [text/plain]
Saving to: ‘requirements.txt.7’

requirements.txt.7  100%[===================>]     197  --.-KB/s    in 0s      

2021-12-29 13:26:26 (7.26 MB/s) - ‘requirements.txt.7’ saved [197/197]



### Loading the Auto MPG Data and Training a Neural Network

We will begin by generating fake data for the Auto MPG dataset we have previously seen. The tabgan library can generate both categorical (textual) and continuous (numeric) data. However, it cannot generate unstructured data, such as the name of the automobile. Car names, such as "AMC Rebel SST" cannot be replicated by the GAN, because every row has a different car name; it is a textual but non-categorical value. 

The following code is similar to what we have seen before. We load the AutoMPG dataset. The tabgan library requires Pandas dataframe to train. Because of this, we keep both the Pandas and Numpy values.

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

COLS_USED = ['cylinders', 'displacement', 'horsepower', 'weight', 
          'acceleration', 'year', 'origin','mpg']
COLS_TRAIN = ['cylinders', 'displacement', 'horsepower', 'weight', 
          'acceleration', 'year', 'origin']

df = df[COLS_USED]

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())


# Split into training and test sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    df.drop("mpg", axis=1),
    df["mpg"],
    test_size=0.20,
    #shuffle=False,
    random_state=42,
)

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

# Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

# Build the neural network
model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(12, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor], verbose=2,epochs=1000)

Epoch 1/1000
10/10 - 2s - loss: 86219.8438 - val_loss: 1912.8424 - 2s/epoch - 168ms/step
Epoch 2/1000
10/10 - 0s - loss: 14992.5957 - val_loss: 9762.6133 - 100ms/epoch - 10ms/step
Epoch 3/1000
10/10 - 0s - loss: 2588.4531 - val_loss: 1899.9395 - 104ms/epoch - 10ms/step
Epoch 4/1000
10/10 - 0s - loss: 2207.9507 - val_loss: 633.3818 - 106ms/epoch - 11ms/step
Epoch 5/1000
10/10 - 0s - loss: 457.9456 - val_loss: 784.1696 - 178ms/epoch - 18ms/step
Epoch 6/1000
10/10 - 0s - loss: 465.8445 - val_loss: 229.7097 - 97ms/epoch - 10ms/step
Epoch 7/1000
10/10 - 0s - loss: 293.9960 - val_loss: 258.0968 - 124ms/epoch - 12ms/step
Epoch 8/1000
10/10 - 0s - loss: 250.8663 - val_loss: 266.4640 - 244ms/epoch - 24ms/step
Epoch 9/1000
10/10 - 0s - loss: 233.4393 - val_loss: 224.1489 - 193ms/epoch - 19ms/step
Epoch 10/1000
10/10 - 0s - loss: 227.7182 - val_loss: 221.9316 - 188ms/epoch - 19ms/step
Epoch 11/1000
10/10 - 0s - loss: 218.8727 - val_loss: 228.2350 - 138ms/epoch - 14ms/step
Epoch 12/1000
10/10 - 0s

We now evaluate the trained neural network to see the RMSE. We will use this trained neural network to compare accuracy between the original data and the GAN-generated data. We will later see that such comparisons can be used for anomaly detection. We can use this technique can be used for security systems. If a neural network trained on original data does not perform well on new data, then the new data may be suspect or fake.

In [4]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 5.337835604017036


### Training a GAN for Auto MPG

Next, we will train the GAN to generate fake data from the original MPG data. There are quite a few options that you can fine-tune for the GAN. The example presented here uses most of the default values. These are the usual hyperparameters that must be tuned for any model and require some experimentation for optimal results. To learn more in-depth about tabgab refer to its paper or this [Medium article](https://towardsdatascience.com/review-of-gans-for-tabular-data-a30a2199342), written by the creator of tabgan.

In [5]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#gen_x, gen_y = GANGenerator().generate_data_pipe(df_x_train, df_y_train, df_x_test, )

gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100, 
               "learning_rate": 0.02, "random_state": 42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,
           gan_params = {"batch_size": 500, "patience": 25, "epochs" : 500,}).generate_data_pipe(df_x_train, df_y_train,
                                          df_x_test, deep_copy=True, only_adversarial=False, use_adversarial=True)



Fitting CTGAN transformers for each column:   0%|          | 0/8 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

### Evaluating the GAN Results

If we display the results, we can see that the GAN-generated data looks similar to the original. Some values, typically whole numbers in the original data, have fractional values in the synthetic data. 

In [6]:
gen_x

,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,8,130.377165,99.373601,3577,18.643748,80,3
1,7,113.257756,97.349653,3679,20.036553,70,3
2,8,125.571491,94.285124,4696,21.566364,75,3
3,7,126.892285,76.207964,3640,10.799983,76,1
4,8,140.185530,87.871277,3547,20.866782,72,2
...,...,...,...,...,...,...,...
591,4,91.000000,68.000000,1985,16.000000,81,3
592,4,108.000000,70.000000,2245,16.900000,82,3
593,4,108.000000,75.000000,2350,16.800000,81,3
594,4,90.000000,75.000000,2108,15.500000,74,2


Finally, we present the synthetic data to the previously trained neural network to see how accurately we can predict the synthetic targets.  As we can see, some RMSE accuracy is lost by going to synthetic data.

In [7]:
# Predict
pred = model.predict(gen_x.values)
score = np.sqrt(metrics.mean_squared_error(pred,gen_y.values))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 11.034968310336694
